In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
df= pd.read_csv('./Data/3_Employee_rentation/Employee_Rentation.csv')
df.head()

,Unnamed: 0,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,1,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,2,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,3,927315.0,4,marketing,14,101000.0,2014-10-13,NaN
3,4,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,5,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24702 entries, 0 to 24701
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   employee_id  24702 non-null  float64
 1   company_id   24702 non-null  int64  
 2   dept         24702 non-null  object 
 3   seniority    24702 non-null  int64  
 4   salary       24702 non-null  float64
 5   join_date    24702 non-null  object 
 6   quit_date    13510 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 1.3+ MB


In [8]:
#parse date
df[['join_date', 'quit_date']] = df[['join_date', 'quit_date']].apply(pd.to_datetime)

# Q1
Assume, for each company, that the headcount starts from zero on 2011/01/23. Estimate employee headcount, for each company, on each day, from 2011/01/24 to 2015/12/13. That is, if by 2012/03/02 2000 people have joined company 1 and 1000 of them have already quit, then company headcount on 2012/03/02 for company 1 would be 1000.You should create a table with 3 columns: day, employee_headcount, company_id.

In [124]:
# create a new table with day, employee_headcount, and company_id
# unique dates table
dates = pd.date_range(start='2011/01/24', end='2015/12/13', freq='D')
dates = pd.DataFrame(dates, columns=['date'])
company = sorted(df['company_id'].unique())
company = pd.DataFrame(company, columns=['company_id'])
# cross join unique dates and company
dates_company = pd.merge(dates, company, how='cross')

# employee headcount table
join_date = df.groupby(['join_date', 'company_id']).size().reset_index()
join_date.columns=['date', 'company_id', 'join']
quit_date = df.groupby(['quit_date', 'company_id']).size().reset_index()
quit_date.columns=['date', 'company_id', 'quit']

headcount = pd.merge(dates_company, join_date, left_on=['date', 'company_id'], right_on=['date', 'company_id'], how='left')
headcount = pd.merge(headcount, quit_date, left_on=['date', 'company_id'], right_on=['date', 'company_id'], how='left')
headcount['join'].fillna(0, inplace=True)
headcount['quit'].fillna(0, inplace=True)
headcount['join_sum'] = headcount.groupby(['company_id'])['join'].cumsum()
headcount['quit_sum'] = headcount.groupby(['company_id'])['quit'].cumsum()
headcount['headcount'] = (headcount['join_sum'] - headcount['quit_sum']).astype(int)
headcount.drop(['join', 'quit', 'join_sum', 'quit_sum'], axis=1, inplace=True)
headcount


,date,company_id,headcount
0,2011-01-24,1,25
1,2011-01-24,2,17
2,2011-01-24,3,9
3,2011-01-24,4,12
4,2011-01-24,5,5
...,...,...,...
21415,2015-12-13,8,468
21416,2015-12-13,9,432
21417,2015-12-13,10,385
21418,2015-12-13,11,4


In [125]:
# create a new table with day, employee_headcount, and company_id using for loop
dates = pd.date_range(start='2011/01/24', end='2015/12/13', freq='D')
company = sorted(df['company_id'].unique())
day = []
company_id = []
headcount = []

for date in dates:
    for c in company:
        day.append(date)
        company_id.append(c)
        join_count = len(df.query('join_date<=@date and company_id==@c'))
        quit_count = len(df.query('quit_date<=@date and company_id==@c'))
        headcount.append(join_count - quit_count)
df_headcount = pd.DataFrame({'date':day, 'company_id':company_id, 'headcount':headcount})
df_headcount


,date,company_id,headcount
0,2011-01-24,1,25
1,2011-01-24,2,17
2,2011-01-24,3,9
3,2011-01-24,4,12
4,2011-01-24,5,5
...,...,...,...
21415,2015-12-13,8,468
21416,2015-12-13,9,432
21417,2015-12-13,10,385
21418,2015-12-13,11,4


# Employee Churn Analysis